## Module 1

I first simply solve numerical module 1 and see that i get the right resistance.

In [ ]:
import numpy as np
from scipy.optimize import root

# --- A. Physical Constants ---
R = 8.314  # Gas Constant (J/mol·K)
T = 300.0  # Temperature (K)
RT = R * T

# --- B. Generate Consistent Rates ---
def generate_consistent_rates(mu_0_values, k_fwd_values):
    """
    Generates thermodynamically consistent kinetic rates for Module 1 
    based on standard chemical potentials (mu_0) and chosen forward rates.
    """
    mu = mu_0_values
    k = k_fwd_values.copy()

    # R1: Ea + S <-> EaS
    Delta_G1_0 = mu['EaS'] - (mu['Ea'] + mu['S'])
    
    # R2: EaS <-> Ea + Na
    Delta_G2_0 = (mu['Ea'] + mu['Na']) - mu['EaS']
    
    # R3: EaS + S <-> EaS2
    Delta_G3_0 = mu['EaS2'] - (mu['EaS'] + mu['S'])

    # Enforce Local Detailed Balance
    k['k1r'] = k['k1f'] * np.exp(Delta_G1_0 / RT)
    k['k2r'] = k['k2f'] * np.exp(Delta_G2_0 / RT)
    k['k3r'] = k['k3f'] * np.exp(Delta_G3_0 / RT)
    
    # Verify Cycle Constraint (R1 + R2 forms the cycle)
    # From paper eq. 68: cycle affinity F = μ_S - μ_Na
    # At equilibrium with mu_S = mu_Na, we need Delta_G_cycle = 0
    Delta_G_cycle_0 = Delta_G1_0 + Delta_G2_0
    
    print(f"ΔG°₁ (R1): {Delta_G1_0:.2f} J/mol")
    print(f"ΔG°₂ (R2): {Delta_G2_0:.2f} J/mol")
    print(f"ΔG°₃ (R3): {Delta_G3_0:.2f} J/mol")
    print(f"ΔG°_cycle (R1+R2, should equal μ_S - μ_Na = {mu['S'] - mu['Na']:.2f}): {Delta_G_cycle_0:.2f} J/mol")
    
    if abs(Delta_G_cycle_0 - (mu['S'] - mu['Na'])) > 1e-9:
        print(f"WARNING: Cycle constraint violated!")
    else:
        print("✓ Cycle constraint satisfied")
    
    return k

# --- Define Parameters ---
mu_0 = {
    'Ea': 2.0,
    'EaS': 3.0,
    'EaS2': 1000000.0,
    'S': 1.0,
    'Na': 1.0  # Equal to S for thermodynamic consistency
}

k_fwd = {'k1f': 6.0, 'k2f': 20.0, 'k3f': 50.0}
k = generate_consistent_rates(mu_0, k_fwd)

# --- Initial Conditions ---
X_initial = np.array([1e-10, 1e-10, 1e-10])
external_boundary_conditions = np.array([10.0, 5.0])  # [S, Na]

# --- Stoichiometric Matrix ---
N_matrix = np.array([
    [-1,  1,  0],  # d(Ea)/dt
    [ 1, -1, -1],  # d(EaS)/dt
    [ 0,  0,  1]   # d(EaS2)/dt
])

def get_reaction_fluxes_from_Y(Y_internal, external, params):
    X_internal = np.exp(Y_internal)
    Ea, EaS, EaS2 = X_internal
    S, Na = external
    
    j_1 = params['k1f'] * S * Ea - params['k1r'] * EaS
    j_2 = params['k2f'] * EaS - params['k2r'] * Na * Ea
    j_3 = params['k3f'] * S * EaS - params['k3r'] * EaS2
    return np.array([j_1, j_2, j_3])

def dYdt(Y_internal, external, params):
    X_internal = np.exp(Y_internal)
    j_fluxes = get_reaction_fluxes_from_Y(Y_internal, external, params)
    dX_dt = np.dot(N_matrix, j_fluxes)
    dY_dt = dX_dt / X_internal
    return dY_dt

def get_steady_state_results(conc_ext, X_init, k_params):
    Y_initial = np.log(X_init)
    sol = root(lambda Y: dYdt(Y, conc_ext, k_params), Y_initial, method='lm', tol=1e-12)
    
    if not sol.success:
        raise ValueError(f"Steady state solver failed: {sol.message}")
    
    Y_ss = sol.x
    X_ss = np.exp(Y_ss)
    j_ss = get_reaction_fluxes_from_Y(Y_ss, conc_ext, k_params)
    
    return X_ss, j_ss

# --- Solve and Display Results ---
X_ss, j_ss = get_steady_state_results(external_boundary_conditions, X_initial, k)

print("\n" + "="*60)
print("MODULE 1: Non-Equilibrium Steady State Results")
print("="*60)
print(f"External: [S] = {external_boundary_conditions[0]:.2f}, [Na] = {external_boundary_conditions[1]:.2f}")
print(f"\nInternal Concentrations at NESS:")
print(f"  [Ea]   = {X_ss[0]:.6f}")
print(f"  [EaS]  = {X_ss[1]:.6f}")
print(f"  [EaS2] = {X_ss[2]:.6f}")

print(f"\nReaction Fluxes at Steady State:")
print(f"  j₁ (Ea + S ⇌ EaS)    = {j_ss[0]:+.6f}")
print(f"  j₂ (EaS ⇌ Ea + Na)   = {j_ss[1]:+.6f}")
print(f"  j₃ (EaS + S ⇌ EaS2)  = {j_ss[2]:+.6f}")

print(f"\n" + "-"*60)
print("VERIFICATION (from paper, eq. 54):")
print("-"*60)
print(f"Cycle condition |j₁ - j₂| = {abs(j_ss[0] - j_ss[1]):.2e} (should be ~0)")
print(f"Dead-end flux j₃ = {j_ss[2]:.2e} (should be ~0)")

# Physical currents (eq. 62 from paper)
# i(1) = [-∇y @ j] where ∇y = [[-1, 0, -1], [0, 1, 0]]
i_S = j_ss[0] + j_ss[2]    # S consumption
i_Na = -j_ss[1]             # Na production

print(f"\nChemostat Currents (paper eq. 62):")
print(f"  i_S  (S → module)   = {i_S:+.6f}")
print(f"  i_Na (module → Na)  = {i_Na:+.6f}")
print(f"  Net current I = j₁  = {j_ss[0]:+.6f}")

print(f"\nMass Balance Check:")
print(f"  |i_S - i_Na| = {abs(i_S - i_Na):.2e} (should be ~0)")
print("="*60)

ΔG°₁ (R1): -2000.00 J/mol
ΔG°₂ (R2): 2000.00 J/mol
ΔG°₃ (R3): 10000.00 J/mol
ΔG°_cycle (R1+R2): 0.00 J/mol
Expected (μ_S - μ_Na): 0.00 J/mol
k1r = 2.690963e+00, k2r = 4.459370e+01, k3r = 5.510837e+01
✓ Cycle constraint satisfied

MODULE 1: Non-Equilibrium Steady State Results
External: [S] = 10.00, [Na] = 5.00

Internal Concentrations at NESS:
  [Ea]   = 7.184005e+305
  [EaS]  = 8.958841e+306
  [EaS2] = 1.625677e+306

Reaction Fluxes at Steady State:
  j₁ (Ea + S ⇌ EaS)    = +1.899612e+307
  j₂ (EaS ⇌ Ea + Na)   = +1.899612e+307
  j₃ (EaS + S ⇌ EaS2)  = -1.786277e+294

------------------------------------------------------------
VERIFICATION (from paper, eq. 54):
------------------------------------------------------------
  |j₁ - j₂| = 9.67e+294 (should be ~0)
  |j₃| = 1.79e+294 (should be ~0)

Chemostat Currents (paper eq. 62):
  i_S  (S → module)   = +1.899612e+307
  i_Na (module → Na)  = +1.899612e+307
  Net current I = j₁  = +1.899612e+307

Mass Balance Check:
  |i_S - i_Na| = 7.8

/var/folders/k2/zvg_gcvj6dz5q4_t5p1n04240000gp/T/ipykernel_2637/178795757.py:90: RuntimeWarning: overflow encountered in exp
  X_internal = np.exp(Y_internal)
/var/folders/k2/zvg_gcvj6dz5q4_t5p1n04240000gp/T/ipykernel_2637/178795757.py:80: RuntimeWarning: overflow encountered in exp
  X_internal = np.exp(Y_internal)
/var/folders/k2/zvg_gcvj6dz5q4_t5p1n04240000gp/T/ipykernel_2637/178795757.py:84: RuntimeWarning: invalid value encountered in scalar subtract
  j_1 = params['k1f'] * S * Ea - params['k1r'] * EaS
/var/folders/k2/zvg_gcvj6dz5q4_t5p1n04240000gp/T/ipykernel_2637/178795757.py:85: RuntimeWarning: invalid value encountered in scalar subtract
  j_2 = params['k2f'] * EaS - params['k2r'] * Na * Ea
/var/folders/k2/zvg_gcvj6dz5q4_t5p1n04240000gp/T/ipykernel_2637/178795757.py:86: RuntimeWarning: invalid value encountered in scalar subtract
  j_3 = params['k3f'] * S * EaS - params['k3r'] * EaS2
/var/folders/k2/zvg_gcvj6dz5q4_t5p1n04240000gp/T/ipykernel_2637/178795757.py:85: RuntimeWarnin

In [22]:
def compute_R_scalar_direct_method(conc_base, X_init, k_params):
    """
    Calculates the 1x1 scalar resistance R = dF/dI using numerical perturbation.
    F = RT * ln(S / Na)
    I = j1
    """
    print("\n=== STARTING SCALAR RESISTANCE CALCULATION (R 1x1) ===")
    
    # 1. Base State Current (I_base)
    X_ss_base, I_base, j_ss_base, _ = get_steady_state_results(conc_base, X_init, k_params)

    # 2. Perturbed State Current (I_new)
    # Apply a small force perturbation dF by changing S concentration.
    perturbation = 1e-7 * RT 
    S_base, Na_base = conc_base
    S_new = S_base * np.exp(perturbation / RT)
    conc_pert = np.array([S_new, Na_base])
    
    X_ss_pert, I_new, _, _ = get_steady_state_results(conc_pert, X_init, k_params)

    # 3. Calculate G and R
    Delta_I = I_new - I_base
    Delta_F = perturbation # The explicit change in force we applied
    
    if np.isclose(Delta_I, 0):
        print("Warning: Delta I is near zero. Resistance is extremely high.")
        return np.inf

    G_scalar = Delta_I / Delta_F
    R_scalar = 1.0 / G_scalar
    
    print("-" * 50)
    print(f"Base Force F_base = {RT * np.log(S_base / Na_base):.4f}")
    print(f"Base Current I_base (j1): {I_base:.6f}")
    print(f"Perturbed Current I_new: {I_new:.6f}")
    print(f"Delta_I: {Delta_I:.2e} | Delta_F: {Delta_F:.2e}")
    print("-" * 50)
    print(f"Conductance G (1/R): {G_scalar:.4f} (mol^2/J·s)")
    print(f"FINAL SCALAR RESISTANCE R(1): {R_scalar:.8f} (J·s/mol^2)")
    print("-" * 50)
    
    return R_scalar

R_value = compute_R_scalar_direct_method(external_boundary_conditions, X_initial, k)


=== STARTING SCALAR RESISTANCE CALCULATION (R 1x1) ===
--------------------------------------------------
Base Force F_base = 1728.8477
Base Current I_base (j1): -3.512077
Perturbed Current I_new: 2.881269
Delta_I: 6.39e+00 | Delta_F: 2.49e-04
--------------------------------------------------
Conductance G (1/R): 25632.8531 (mol^2/J·s)
FINAL SCALAR RESISTANCE R(1): 0.00003901 (J·s/mol^2)
--------------------------------------------------


In [28]:
# Direct analytics from the paper
def compute_R_analytical(conc_ext, X_init, k_params):
    """
    Calculates the analytical scalar resistance R(1) = r1 + r2 at the NESS
    using the steady-state concentrations found by the solver.
    """
    print("\n=== STARTING ANALYTICAL RESISTANCE CALCULATION ===")

    # 1. Get NESS concentrations and fluxes
    try:
        X_ss, I_net, j_ss, _ = get_steady_state_results(conc_ext, X_init, k_params)
    except ValueError as e:
        print(f"Error retrieving NESS for analytical calculation: {e}")
        return np.nan

    Ea, EaS, EaS2 = X_ss
    S, Na = conc_ext
    RT = R * T
    
    # --- R1: Ea + S <-> EaS ---
    v1_fwd = k_params['k1f'] * S * Ea
    v1_rev = k_params['k1r'] * EaS
    print(S, Ea, EaS, v1_fwd, v1_rev)

    
    # Calculate r1
    r1 = RT * (1.0 / v1_fwd + 1.0 / v1_rev)
    
    # --- R2: EaS <-> Ea + Na ---
    v2_fwd = k_params['k2f'] * EaS
    v2_rev = k_params['k2r'] * Na * Ea
    
    # Calculate r2
    r2 = RT * (1.0 / v2_fwd + 1.0 / v2_rev)
    
    # --- R3: EaS + S <-> EaS2 ---
    # This resistance r3 is forced to detailed balance (j3=0) by the stoichiometry, 
    # making it numerically very large or infinite. It is excluded from the cycle resistance R(1).
    
    # --- Total Analytical Resistance ---
    R_analytical = r1 + r2
    
    print("-" * 50)
    print(f"r1 (RT * (1/v1f + 1/v1r)): {r1:.4f}")
    print(f"r2 (RT * (1/v2f + 1/v2r)): {r2:.4f}")
    print(f"ANALYTICAL SCALAR RESISTANCE R(1) = r1 + r2: {R_analytical:.4f} (J·s/mol^2)")
    print("-" * 50)
    
    return R_analytical


R_analytical_value = compute_R_analytical(external_boundary_conditions, X_initial, k)
print(f"Numerical R(1): {R_value:.4f}")
print(f"Analytical R(1): {R_analytical_value:.4f}")



=== STARTING ANALYTICAL RESISTANCE CALCULATION ===
10.0 -0.14581031960561927 -1.0664837179910833 -8.748619176337156 -5.236542133350588
--------------------------------------------------
r1 (RT * (1/v1f + 1/v1r)): -761.4031
r2 (RT * (1/v2f + 1/v2r)): -256.9209
ANALYTICAL SCALAR RESISTANCE R(1) = r1 + r2: -1018.3240 (J·s/mol^2)
--------------------------------------------------
Numerical R(1): 0.0000
Analytical R(1): -1018.3240


In [26]:
k

{'k1f': 6.0,
 'k2f': 20.0,
 'k3f': 50.0,
 'k1r': 4.910100402859006,
 'k2r': 24.43941878054623,
 'k3r': 42.591339830345184}